In [1]:
import numpy as np
import pandas as pd
from dataLoad import PulsarData
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Read in the pulsar data from the dataLoad.py script

In [2]:
features_data = PulsarData('HTRU_2').features
targets_data = PulsarData('HTRU_2').targets

Shuffle and split the data into training and test groups with 3:1 split

In [3]:
train_features_data, test_features_data, train_targets_data, test_targets_data  =  train_test_split( features_data, 
                                                        targets_data, test_size=0.25, random_state=42)

Fit a gradient boosting classifier with 100 decision stumps as weak learners

In [20]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5, 
                                 max_depth=1, random_state=0).fit(train_features_data, train_targets_data)

Compare the classified data to the test set - returns percentage match

In [21]:
clf.score(test_features_data, test_targets_data)

0.9787709497206704